In [ ]:
!pip install standard-precip
!pip install geopandas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
climate_data = pd.read_csv('/content/drive/MyDrive/MCM Practicum/Implementation/datasets/district_climate_data_v2.csv')
climate_data.head()

,year_month,PS,QV2M,RH2M,WD2M,WS2M,WD10M,WS10M,GWETTOP,GWETPROF,...,WS2M_RANGE,PRECTOTCORR,WS10M_RANGE,PRECTOTCORR_SUM,district,month_number,day_number,year,date,month_name
0,200101,94.46,5.68,34.00,81.75,1.52,76.00,2.37,0.30,0.6,...,3.54,0.01,4.55,0.0,Ahmednagar,1,1,2001,01-01-2001,January
1,200102,94.37,3.72,20.81,327.81,1.87,331.56,2.89,0.16,0.6,...,4.63,0.00,6.45,0.0,Ahmednagar,2,1,2001,02-01-2001,February
2,200103,94.26,6.47,31.31,284.75,2.63,286.69,3.66,0.14,0.6,...,5.84,0.03,7.55,0.0,Ahmednagar,3,1,2001,03-01-2001,March
3,200104,94.14,7.45,26.31,345.19,2.10,343.81,2.90,0.18,0.6,...,5.00,0.16,6.81,0.0,Ahmednagar,4,1,2001,04-01-2001,April
4,200105,93.89,11.47,42.81,272.44,4.43,272.44,5.75,0.17,0.6,...,6.72,0.18,9.06,0.0,Ahmednagar,5,1,2001,05-01-2001,May


In [ ]:
climate_data.dtypes

year_month           int64
PS                 float64
QV2M               float64
RH2M               float64
WD2M               float64
WS2M               float64
WD10M              float64
WS10M              float64
GWETTOP            float64
GWETPROF           float64
GWETROOT           float64
WS2M_MAX           float64
WS2M_MIN           float64
WS10M_MAX          float64
WS10M_MIN          float64
WS2M_RANGE         float64
PRECTOTCORR        float64
WS10M_RANGE        float64
PRECTOTCORR_SUM    float64
district            object
month_number         int64
day_number           int64
year                 int64
date                object
month_name          object
dtype: object

### Features Description

|PARAMETERS      |DESCRIPTION                                       |
|--              |--                                                |
|PS              |   Surface Pressure (kPa)                         |
|TS              |   Earth Skin Temperature (C)                     |
|T2M             |   Temperature at 2 Meters (C)                    |
|QV2M            |   Specific Humidity at 2 Meters (g/kg)           |
|RH2M            |   Relative Humidity at 2 Meters (%)              |   
|WD2M            |   Wind Direction at 2 Meters (Degrees)           |   
|WS2M            |   Wind Speed at 2 Meters (m/s)                   |
|T2MDEW          |   Dew/Frost Point at 2 Meters (C)                |
|T2MWET          |   Wet Bulb Temperature at 2 Meters (C)           |
|GWETTOP         |   Surface Soil Wetness (1)                       |
|GWETPROF        |   Profile Soil Moisture (1)                      |
|GWETROOT        |   Root Zone Soil Wetness (1)                     |
|WS2M_MAX        |   Wind Speed at 2 Meters Maximum (m/s)           |
|WS2M_MIN        |   Wind Speed at 2 Meters Minimum (m/s)           |
|WS2M_RANGE      |   Wind Speed at 2 Meters Range (m/s)             |
|PRECTOTCORR     |   Precipitation Corrected (mm/day)               |
|PRECTOTCORR_SUM |   Precipitation Corrected Sum (mm)               |

In [ ]:
fig = px.line(climate_data[['year', 'PRECTOTCORR']].groupby(['year']).mean().reset_index(),
              x='year',
              y="PRECTOTCORR",
              labels={'PRECTOTCORR': 'Precipitation Corrected (mm/day)'},
              title="Annual Precipitation in Maharashtra")

fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
    )
)
fig.show()

# Calculate Standard Precipitation Index

In [ ]:
from standard_precip.spi import SPI
from standard_precip.utils import plot_index

spi = SPI()

In [ ]:
climate_data.district.unique()

array(['Ahmednagar', 'Akola', 'Amravati', 'Aurangabad', 'Beed',
       'Bhandara', 'Buldhana', 'Chandrapur', 'Dhule', 'Gadchiroli',
       'Gondia', 'Hingoli', 'Jalgaon', 'Jalna', 'Kolhapur', 'Latur',
       'Mumbai City', 'Mumbai Suburban', 'Nagpur', 'Nanded', 'Nandurbar',
       'Nashik', 'Osmanabad', 'Palghar', 'Parbhani', 'Pune', 'Raigad',
       'Ratnagiri', 'Sangli', 'Satara', 'Sindhudurg', 'Solapur', 'Thane',
       'Wardha', 'Washim', 'Yavatmal'], dtype=object)

In [ ]:
climate_data['year_month_formatted'] = pd.to_datetime(climate_data['year_month'].astype(str)+'01', format="%Y%m%d").dt.to_period("M")
climate_data['year_month_formatted'] = climate_data['year_month_formatted'].astype(str)

In [ ]:
# Define the conditions and corresponding classes
def classify_drought_severity(spi_values):
    # Use np.where() to assign the drought severity classes
    drought_classes = np.where(spi_values >= 2.0, 'Extremely Wet',
                    np.where(((spi_values >= 1.5) & (spi_values <= 1.99)), 'Very Wet',
                    np.where(((spi_values >= 1.0) & (spi_values <= 1.49)), 'Moderately Wet',
                    np.where(((spi_values >= -0.99) & (spi_values <= 0.99)), 'Near Normal',
                    np.where(((spi_values >= -1.49) & (spi_values <= -1.0)), 'Moderately Dry',
                    np.where(((spi_values >= -1.99) & (spi_values <= -1.5)), 'Severely Dry', 'Extremely Dry'))))))

    return drought_classes.item()

In [ ]:
single_dist = pd.DataFrame()
all_district_data = pd.DataFrame()

for dist in climate_data.district.unique():
    single_dist = climate_data.query(f"""district=='{dist}'""").copy()

    spi1 = spi.calculate(single_dist, 'year_month_formatted', 'PRECTOTCORR', freq="M", scale=1, fit_type="lmom", dist_type="gam").rename(columns={'PRECTOTCORR_calculated_index': 'spi1'})
    spi1["spi1"].fillna(spi1["spi1"].mean(), inplace=True)
    spi1["spi1_color"] = np.where(spi1["spi1"]<0, 'red', 'green')
    spi1["spi1_drought_category"] = spi1["spi1"].map(lambda x: classify_drought_severity(x))

    spi3 = spi.calculate(single_dist, 'year_month_formatted', 'PRECTOTCORR', freq="M", scale=3, fit_type="lmom", dist_type="gam").rename(columns={'PRECTOTCORR_scale_3_calculated_index': 'spi3'})

    spi3["spi3"].fillna(spi3["spi3"].mean(), inplace=True)
    spi3["spi3_color"] = np.where(spi3["spi3"]<0, 'red', 'green')
    spi3["spi3_drought_category"] = spi3["spi3"].map(lambda x: classify_drought_severity(x))
    # spi3 = spi3.drop(columns=['PRECTOTCORR_scale_3'], axis=1)

    spi6 = spi.calculate(single_dist, 'year_month_formatted', 'PRECTOTCORR', freq="M", scale=6, fit_type="lmom", dist_type="gam").rename(columns={'PRECTOTCORR_scale_6_calculated_index': 'spi6'})
    spi6["spi6"].fillna(spi6["spi6"].mean(), inplace=True)
    spi6["spi6_color"] = np.where(spi6["spi6"]<0, 'red', 'green')
    spi6["spi6_drought_category"] = spi6["spi6"].map(lambda x: classify_drought_severity(x))
    # spi6 = spi6.drop(columns=['PRECTOTCORR_scale_6'], axis=1)

    single_dist = single_dist.drop(columns=['PRECTOTCORR_scale_3', 'PRECTOTCORR_scale_6'], axis=1)
    single_dist.reset_index(drop=True, inplace=True)
    spi_data = pd.concat([single_dist,
                          spi1[['spi1', 'spi1_color', 'spi1_drought_category']],
                          spi3[['spi3', 'spi3_color', 'spi3_drought_category']],
                          spi6[['spi6', 'spi6_color', 'spi6_drought_category']]], axis=1)

    all_district_data = pd.concat([all_district_data, spi_data], axis=0, ignore_index=True)

In [ ]:
climate_data.shape, all_district_data.shape

((9072, 26), (9072, 35))

In [ ]:
all_district_data

,year_month,PS,QV2M,RH2M,WD2M,WS2M,WD10M,WS10M,GWETTOP,GWETPROF,...,year_month_formatted,spi1,spi1_color,spi1_drought_category,spi3,spi3_color,spi3_drought_category,spi6,spi6_color,spi6_drought_category
0,200101,94.46,5.68,34.00,81.75,1.52,76.00,2.37,0.30,0.60,...,2001-01,0.046818,green,Near Normal,0.038293,green,Near Normal,0.008836,green,Near Normal
1,200102,94.37,3.72,20.81,327.81,1.87,331.56,2.89,0.16,0.60,...,2001-02,-0.180012,red,Near Normal,0.038293,green,Near Normal,0.008836,green,Near Normal
2,200103,94.26,6.47,31.31,284.75,2.63,286.69,3.66,0.14,0.60,...,2001-03,-0.293911,red,Near Normal,-0.712338,red,Near Normal,0.008836,green,Near Normal
3,200104,94.14,7.45,26.31,345.19,2.10,343.81,2.90,0.18,0.60,...,2001-04,0.017822,green,Near Normal,-0.642136,red,Near Normal,0.008836,green,Near Normal
4,200105,93.89,11.47,42.81,272.44,4.43,272.44,5.75,0.17,0.60,...,2001-05,-0.462521,red,Near Normal,-0.800249,red,Near Normal,0.008836,green,Near Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9067,202108,96.79,18.92,83.50,285.88,2.36,285.81,3.44,0.84,0.82,...,2021-08,-0.666158,red,Near Normal,-0.053367,red,Near Normal,-0.001607,red,Near Normal
9068,202109,96.76,19.47,87.69,269.12,2.00,268.12,3.01,0.92,0.91,...,2021-09,1.779563,green,Very Wet,0.726030,green,Near Normal,0.971439,green,Near Normal
9069,202110,97.29,15.75,79.44,37.75,1.36,37.50,2.16,0.90,0.91,...,2021-10,1.373699,green,Moderately Wet,1.501042,green,Very Wet,1.259494,green,Moderately Wet
9070,202111,97.54,13.61,81.56,69.50,1.66,69.00,2.65,0.80,0.79,...,2021-11,0.223089,green,Near Normal,2.031569,green,Extremely Wet,1.151909,green,Moderately Wet


In [ ]:
all_district_data.columns

Index(['year_month', 'PS', 'QV2M', 'RH2M', 'WD2M', 'WS2M', 'WD10M', 'WS10M',
       'GWETTOP', 'GWETPROF', 'GWETROOT', 'WS2M_MAX', 'WS2M_MIN', 'WS10M_MAX',
       'WS10M_MIN', 'WS2M_RANGE', 'PRECTOTCORR', 'WS10M_RANGE',
       'PRECTOTCORR_SUM', 'district', 'month_number', 'day_number', 'year',
       'date', 'month_name', 'year_month_formatted', 'spi1', 'spi1_color',
       'spi1_drought_category', 'spi3', 'spi3_color', 'spi3_drought_category',
       'spi6', 'spi6_color', 'spi6_drought_category'],
      dtype='object')

In [ ]:
all_district_data.to_csv('/content/drive/MyDrive/MCM Practicum/Implementation/datasets/district_spi_data.csv', index=False, header=True)